# Boîte à outils

In [12]:
# FrenchLefffLemmatizer à installer si votre environnement ne le contient pas : 
# pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
import nltk

# Télécharger les packages avec les lignes suivantes si vous ne les avez pas sur votre machine : 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /Users/perrine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/perrine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/perrine/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
from pathlib import Path
from collections import defaultdict
import nltk
from nltk.stem import WordNetLemmatizer
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import wordnet
import spacy

In [14]:
# Lemmatiseur anglais
lemmatizer_en = WordNetLemmatizer()
# Lemmatiseur français
lemmatizer_fr = FrenchLefffLemmatizer()

In [15]:
# Parseur Spacy (facultatif)
nlp = spacy.load("fr_core_news_md")

# Lecture des corpus prétraités

In [16]:
# Les chemins vers les corpus
path_europarl = Path("../data/clean/Europarl_en_fr_clean/")
path_emea = Path("../data/clean/Emea_en_fr_clean/")

# Créer un dictionnaire pour enregistrer les corpus avec leurs noms de fichiers
corpus_list = defaultdict(list)

# Parcourir chaque fichier de l'Europarl
for file in path_europarl.glob('*tok.true.clean*'):
    with open(file, 'r') as file:
        sents_brut = file.read()
        # Extraire seulement le nom de fichiers comme clés 
        corpus_list[file.name.split('/')[-1]] = sents_brut.split('\n')

# Parcourir chaque fichier de l'EMEA
for file in path_emea.glob('*tok.true.clean*'):
    with open(file, 'r') as file:
        sents_brut = file.read()
        # Extraire seulement le nom de fichiers comme clés 
        corpus_list[file.name.split('/')[-1]] = sents_brut.split('\n')

# Lemmatisation

In [17]:
# Fonction pour lemmatiser un texte
def lemmatize_text(text: str, lang: str) -> str:
    """
    Lemmatisation d'un texte donné dans la langue spécifiée.

    Cette fonction lemmatise un texte en utilisant NLTK pour l'anglais et Spacy ou NLTK pour le français. 
    Pour l'anglais, les parties du discours (POS tags) sont utilisées pour une lemmatisation plus précise. 
    Pour le français, la lemmatisation de base est effectuée avec lemmatizer_fr de FrenchLefffLemmatizer. 
    
    Args:
        text (str): Le texte à lemmatiser.
        lang (str): La langue du texte. Peut être 'en' pour l'anglais ou 'fr' pour le français.

    Returns:
        str: Le texte lemmatisé.

    Raises:
        ValueError: Si la langue spécifiée n'est ni 'en' ni 'fr'.
    """

    # Lemmatisation de chaque mot avec son POS tag
    lemmatized_words = []

    if lang == "en":
        
        # Tokeniser le texte en mots
        words = nltk.word_tokenize(text)
    
        # Obtenir les POS tags des mots
        pos_tags = nltk.pos_tag(words)
        
        for word, pos_tag in pos_tags:
            # Convertir les POS tags Penn Treebank en format WordNet
            pos_tag = pos_tag[0].lower()
            pos_tag = pos_tag if pos_tag in ['a', 'n', 'v'] else 'n'  # Convertir les POS tags inconnus en 'n' (nom)
    
            
            # Lemmatisation du mot avec son POS tag
            lemma = lemmatizer_en.lemmatize(word, pos=pos_tag)
            lemmatized_words.append(lemma)

    elif lang == "fr":

        # Lemmatisation avec spacy: beaucoup beaucoup plus précise mais nécessite une machine puissante 
        
        # doc = nlp(text)
    
        # for token in doc:
        #     # Lemmatisation du mot avec son POS tag
        #     lemma = token.lemma_
        #     lemmatized_words.append(lemma)
        
        
        # Tokeniser le texte en mots
        words = nltk.word_tokenize(text)
            
        # Lemmatisation du mot avec son POS tag
        lemmatized_words = [lemmatizer_fr.lemmatize(word) for word in words]
        
    # Rejoindre les mots lemmatisés en une seule chaîne de caractères
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

In [18]:
# Créer un dictionnaire pour stocker les phrases lemmatisées
corpus_lemmatized = defaultdict(list)

# Parcourir chaque corpus
for k, v in corpus_list.items(): # k: nom de fichier, v: liste de phrases
    # Créer une liste pour stocker les phrases lemmatisées
    sents_lemmatized: list = []

    # Pour les textes en anglais
    if k.endswith('en'):
        # Lemmatiser chaque phrase
        for sent in v:
            # Lemmatiser la phrase en anglais
            sent_lemmatized = lemmatize_text(sent, 'en')
            # Ajouter la phrase lemmatisée à la liste
            sents_lemmatized.append(sent_lemmatized)
        # Stocker la liste de phrases lemmatisées dans le dictionnaire
        corpus_lemmatized[k] = sents_lemmatized

    # Pour les textes en français
    elif k.endswith('fr'):
        # Lemmatiser chaque phrase
        for sent in v:
            # Lemmatiser la phrase en français
            sent_lemmatized = lemmatize_text(sent, 'fr')
            # Ajouter la phrase lemmatisée à la liste
            sents_lemmatized.append(sent_lemmatized)
        # Stocker la liste de phrases lemmatisées dans le dictionnaire
        corpus_lemmatized[k] = sents_lemmatized

In [19]:
# Exporter les corpus lemmatisés
for k, v in corpus_lemmatized.items():
    with open(f'../data/clean/lemmatised_spacy/lemme_{k}', 'w', encoding='utf-8') as file:
        file.write('\n'.join(v))